# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 3 2022 1:28PM,247762,15,PNC381188A,"Person & Covey, Inc.",Released
1,Oct 3 2022 1:28PM,247762,15,PNC381369A,"Person & Covey, Inc.",Released
2,Oct 3 2022 1:28PM,247762,15,PNC381376A,"Person & Covey, Inc.",Released
3,Oct 3 2022 1:28PM,247762,15,PNC381382A,"Person & Covey, Inc.",Released
4,Oct 3 2022 1:28PM,247762,15,PNC381455A,"Person & Covey, Inc.",Released
5,Oct 3 2022 1:28PM,247762,15,PNC384442A,"Person & Covey, Inc.",Released
6,Oct 3 2022 1:28PM,247762,15,PNC384454A,"Person & Covey, Inc.",Released
7,Oct 3 2022 1:28PM,247762,15,PNC384565A,"Person & Covey, Inc.",Released
8,Oct 3 2022 1:22PM,247754,10,0086041291,ISDIN Corporation,Released
9,Oct 3 2022 1:22PM,247754,10,0086041290,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,247757,Released,3
29,247759,Executing,1
30,247759,Released,2
31,247761,Released,1
32,247762,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247756,NaN,NaN,1.0
247757,NaN,NaN,3.0
247759,NaN,1.0,2.0
247761,NaN,NaN,1.0
247762,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247684,0.0,0.0,15.0
247692,0.0,0.0,1.0
247694,0.0,0.0,1.0
247706,0.0,0.0,1.0
247707,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247684,0,0,15
247692,0,0,1
247694,0,0,1
247706,0,0,1
247707,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247684,0,0,15
1,247692,0,0,1
2,247694,0,0,1
3,247706,0,0,1
4,247707,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247684,,,15
1,247692,,,1
2,247694,,,1
3,247706,,,1
4,247707,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc."
8,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation
67,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation
124,Oct 3 2022 1:21PM,247761,12,LF of America Corp.
125,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc."
126,Oct 3 2022 1:10PM,247759,19,"Graystone, LLC"
129,Oct 3 2022 1:01PM,247757,10,Emerginnova
132,Oct 3 2022 1:00PM,247756,10,"Snap Medical Industries, LLC"
133,Oct 3 2022 12:55PM,247755,19,ACG North America LLC
134,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc.",,,8
1,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation,,,59
2,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation,,,57
3,Oct 3 2022 1:21PM,247761,12,LF of America Corp.,,,1
4,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc.",,,1
5,Oct 3 2022 1:10PM,247759,19,"Graystone, LLC",,1,2
6,Oct 3 2022 1:01PM,247757,10,Emerginnova,,,3
7,Oct 3 2022 1:00PM,247756,10,"Snap Medical Industries, LLC",,,1
8,Oct 3 2022 12:55PM,247755,19,ACG North America LLC,,,1
9,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc.",8,,
1,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation,59,,
2,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation,57,,
3,Oct 3 2022 1:21PM,247761,12,LF of America Corp.,1,,
4,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc.",1,,
5,Oct 3 2022 1:10PM,247759,19,"Graystone, LLC",2,1,
6,Oct 3 2022 1:01PM,247757,10,Emerginnova,3,,
7,Oct 3 2022 1:00PM,247756,10,"Snap Medical Industries, LLC",1,,
8,Oct 3 2022 12:55PM,247755,19,ACG North America LLC,1,,
9,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc.",8,,
1,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation,59,,
2,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation,57,,
3,Oct 3 2022 1:21PM,247761,12,LF of America Corp.,1,,
4,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc.",8,NaN,NaN
1,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation,59,NaN,NaN
2,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation,57,NaN,NaN
3,Oct 3 2022 1:21PM,247761,12,LF of America Corp.,1,NaN,NaN
4,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc.",8,0.0,0.0
1,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation,59,0.0,0.0
2,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation,57,0.0,0.0
3,Oct 3 2022 1:21PM,247761,12,LF of America Corp.,1,0.0,0.0
4,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3220483,219,3.0,1.0
12,247761,1,0.0,0.0
15,1238751,33,0.0,0.0
16,743135,11,0.0,0.0
19,991006,31,23.0,0.0
20,247719,1,0.0,0.0
21,247694,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3220483,219,3.0,1.0
1,12,247761,1,0.0,0.0
2,15,1238751,33,0.0,0.0
3,16,743135,11,0.0,0.0
4,19,991006,31,23.0,0.0
5,20,247719,1,0.0,0.0
6,21,247694,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,219,3.0,1.0
1,12,1,0.0,0.0
2,15,33,0.0,0.0
3,16,11,0.0,0.0
4,19,31,23.0,0.0
5,20,1,0.0,0.0
6,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,219.0
1,12,Released,1.0
2,15,Released,33.0
3,16,Released,11.0
4,19,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,0.0,0.0,0.0,23.0,0.0,0.0
Released,219.0,1.0,33.0,11.0,31.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,23.0,0.0,0.0
2,Released,219.0,1.0,33.0,11.0,31.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,23.0,0.0,0.0
2,Released,219.0,1.0,33.0,11.0,31.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()